In [1]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate, ChatPromptTemplate
from langchain.schema import StrOutputParser
from operator import itemgetter
from langchain.schema.runnable import RunnablePassthrough
from langchain.callbacks import get_openai_callback
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
model = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.8)

In [5]:
prompt1 = PromptTemplate.from_template("what is the city {person} is from?")
prompt2 = PromptTemplate.from_template("what country is the city {city} in? respond in {language}")

In [6]:
chain1 = prompt1 | model | StrOutputParser()
chain2 = {"city": chain1, "language": itemgetter("language") } | prompt2 | model | StrOutputParser()
chain2.invoke({"person": "Donald Trump", "language": "Vietnamese"})

'Thành phố mà Donald Trump đến từ là New York, tiểu bang New York, thuộc nước Mỹ.'

In [7]:


prompt3 = PromptTemplate.from_template("generate a {attribute} color with the {intensity} saturation. Return the name of the color and nothing else.")
prompt4 = PromptTemplate.from_template("what is the fruit of color: {color}. Return the name of the fruit and nothing else.")
prompt5 = PromptTemplate.from_template("what is the name of the country with a flag that has the color: {color}. Return the name of the country and nothing else.")
prompt6 = PromptTemplate.from_template("what is the color of the {fruit} and the flag of {country}")

model_parser = model | StrOutputParser()

In [25]:
color_generator = {"attribute": RunnablePassthrough()} | prompt3 | {"color": model_parser}
# we can use itemgetter instead of RunnablePassthrough
# color_generator = {"attribute": itemgetter("attribute")} | prompt3 | {"color": model_parser}
color_to_fruit = prompt4 | model_parser
color_to_country = prompt5 | model_parser
question_generator = color_generator | {"fruit": color_to_fruit, "country": color_to_country} | prompt6
# question_generator.invoke({"attribute": "warm"})
question_generator.invoke("warm")

StringPromptValue(text='what is the color of the Cherry and the flag of Kazakhstan')

In [19]:
prompt = question_generator.invoke("warm")
model.invoke(prompt)

AIMessage(content='The color of a cherry is typically red. \n\nThe flag of Chile consists of two horizontal bands of white and red with a blue square in the top-left corner. Inside the blue square, there is a five-pointed white star.')

In [21]:
""" This is example of RunnablePassThrough"""

def fake_llm(prompt: str) -> str: # Fake LLM for the example
    return "completion"

runnable = {
    'llm1':  fake_llm,
    'llm2':  fake_llm,
} | RunnablePassthrough.assign(
    total_chars=lambda inputs: len(inputs['llm1'] + inputs['llm2']))

runnable.invoke('hello')

{'llm1': 'completion', 'llm2': 'completion', 'total_chars': 20}

Branching & Merging
You may want the output of one component to be processed by 2 or more other components. RunnableMaps let you split or fork the chain so multiple components can process the input in parallel. Later, other components can join or merge the results to synthesize a final response. This type of chain creates a computation graph that looks like the following:

     Input
      / \
     /   \
 Branch1 Branch2
     \   /
      \ /
    Combine

In [3]:
initial_input = ChatPromptTemplate.from_template("generate an argument about: {input}") | model | StrOutputParser() | {"base_response": RunnablePassthrough()}
positive = ChatPromptTemplate.from_template("generate a positive aspects of {base_response}") | model | StrOutputParser()
negative = ChatPromptTemplate.from_template("generate a negative aspects of {base_response}") | model | StrOutputParser()
combined = ChatPromptTemplate.from_messages([("ai", "{original_response}"), ("human", "Pros:\n{result_positive}\n\nCons:\n{result_negative}"),
                                              "system", "Generate a final response given the critique"]) | model | StrOutputParser()
chain = initial_input | {"result_positive": positive, "result_negative": negative, "original_response": itemgetter("base_response")} | combined


In [ ]:
with get_openai_callback() as callback:
    for s in chain.stream({"input": """investing tremendous amount of time into learning and practicing 
                           an 10-month old framework called LangChain that enables building AI-powered apps fast and easy."""}):
        print(s, end="")
    print(f"\nHere is the cost breakdown for this call:\n{callback}")

In [ ]:
with get_openai_callback() as callback:
    result = chain.invoke({"input": """investing tremendous amount of time into learning and practicing 
                           an 10-month old framework called LangChain that enables building AI-powered apps fast and easy."""})
    print(f"\nHere is the cost breakdown for this call:\n{callback}")